Enter State Farm

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
from shutil import copyfile
from shutil import move
import pandas as pd
import random

In [4]:

path = "data/"
#path = "data/sample/"

In [5]:
batch_size=64

# Create Validation Set
Determine number of files


In [9]:
# point to the 'driver_imgs_list.csv'
## it's in data/driver_imgs_list.csv   (we're in data/train)
lookup = '../driver_imgs_list.csv'

In [6]:
%cd data/train

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data/train


In [8]:
## make the 10 c* directories
for d in glob('c?'):
    os.mkdir('../valid/'+d)


In [11]:
# read in the 'driver_imgs_list.csv' file
df = pd.read_csv(lookup)

In [13]:
unique_drivers = np.unique(df[['subject']].values)
unique_drivers

array(['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039',
       'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064',
       'p066', 'p072', 'p075', 'p081'], dtype=object)

In [17]:
## there are 26 in total.  Randomly select 5
driver_names_for_validation = random.sample(unique_drivers, 5)
driver_names_for_validation

['p047', 'p041', 'p035', 'p045', 'p064']

In [18]:

# filter df by selecting rows with the subjects we want
df = df[df['subject'].isin(driver_names_for_validation)]

In [21]:
# do the move here  (currently in data/train/ ):
i = 0
for index, row in df.iterrows():
    ##to_move = train_dir + '/' +  row['classname'] + '/' + row['img']
    to_move = row['classname'] + '/' + row['img']
    move_to = '../valid/' +  row['classname']
    print (to_move + ' -> ' + move_to) 
    ##i = i + 1
    ##move_to = val_dir + '/' +  row['classname']
    ##print move_to
    move(to_move, move_to)

c0/img_66660.jpg -> ../valid/c0
c0/img_89362.jpg -> ../valid/c0
c0/img_13584.jpg -> ../valid/c0
c0/img_4400.jpg -> ../valid/c0
c0/img_48340.jpg -> ../valid/c0
c0/img_37109.jpg -> ../valid/c0
c0/img_48482.jpg -> ../valid/c0
c0/img_3509.jpg -> ../valid/c0
c0/img_59446.jpg -> ../valid/c0
c0/img_42283.jpg -> ../valid/c0
c0/img_5869.jpg -> ../valid/c0
c0/img_93090.jpg -> ../valid/c0
c0/img_55476.jpg -> ../valid/c0
c0/img_36549.jpg -> ../valid/c0
c0/img_78110.jpg -> ../valid/c0
c0/img_99289.jpg -> ../valid/c0
c0/img_51076.jpg -> ../valid/c0
c0/img_51232.jpg -> ../valid/c0
c0/img_11067.jpg -> ../valid/c0
c0/img_19302.jpg -> ../valid/c0
c0/img_55729.jpg -> ../valid/c0
c0/img_76704.jpg -> ../valid/c0
c0/img_7044.jpg -> ../valid/c0
c0/img_101223.jpg -> ../valid/c0
c0/img_34741.jpg -> ../valid/c0
c0/img_90390.jpg -> ../valid/c0
c0/img_38463.jpg -> ../valid/c0
c0/img_62061.jpg -> ../valid/c0
c0/img_74744.jpg -> ../valid/c0
c0/img_91123.jpg -> ../valid/c0
c0/img_70438.jpg -> ../valid/c0
c0/img_1603

In [24]:
%cd ..

/home/ubuntu/courses/deeplearning1/kaggle/statefarm


## Setup batches

In [6]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 18592 images belonging to 10 classes.
Found 3832 images belonging to 10 classes.


In [38]:
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)

Found 79726 images belonging to 1 classes.


In [44]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18592 images belonging to 10 classes.
Found 3832 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


## Single Conv Layer

In [29]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [ ]:
model = conv1(batches)

Epoch 1/2
18592/18592 [==============================] - 298s - loss: 0.2363 - acc: 0.9406 - val_loss: 1.6369 - val_acc: 0.5144
Epoch 2/2
18592/18592 [==============================] - 286s - loss: 0.0150 - acc: 0.9988 - val_loss: 1.5286 - val_acc: 0.5517
Epoch 1/4
18592/18592 [==============================] - 301s - loss: 0.0042 - acc: 0.9998 - val_loss: 1.7641 - val_acc: 0.5423
Epoch 2/4
18592/18592 [==============================] - 288s - loss: 0.0023 - acc: 1.0000 - val_loss: 1.5856 - val_acc: 0.5739
Epoch 3/4
18528/18592 [============================>.] - ETA: 0s - loss: 0.0015 - acc: 1.0000

In [ ]:
model.save_weights(path+'models/conv1.h5')

## 4 conv/pooling pairs + dropout

In [8]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 18592 images belonging to 10 classes.


In [13]:

def get_conv_pool_dropout_layers(p):
    return [
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
    ]

In [18]:
## changed the dropout rate from 0.5 (typical starting point) to 0.3 to reduce underfitting  (when the val_acc > acc)
p=0.3

In [19]:
model = Sequential(get_conv_pool_dropout_layers(p))

In [20]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
18592/18592 [==============================] - 301s - loss: 2.0552 - acc: 0.3462 - val_loss: 2.1567 - val_acc: 0.2547
Epoch 2/2
18592/18592 [==============================] - 306s - loss: 1.1704 - acc: 0.6031 - val_loss: 1.1278 - val_acc: 0.6263


In [22]:
model.optimizer.lr=0.001
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/10
18592/18592 [==============================] - 301s - loss: 0.8033 - acc: 0.7359 - val_loss: 0.8971 - val_acc: 0.7427
Epoch 2/10
18592/18592 [==============================] - 303s - loss: 0.6006 - acc: 0.8030 - val_loss: 0.8424 - val_acc: 0.7711
Epoch 3/10
18592/18592 [==============================] - 305s - loss: 0.4786 - acc: 0.8490 - val_loss: 0.9147 - val_acc: 0.7323
Epoch 4/10
18592/18592 [==============================] - 302s - loss: 0.3842 - acc: 0.8789 - val_loss: 0.7048 - val_acc: 0.8346
Epoch 5/10
18592/18592 [==============================] - 305s - loss: 0.3205 - acc: 0.8997 - val_loss: 0.7307 - val_acc: 0.8173
Epoch 6/10
18592/18592 [==============================] - 308s - loss: 0.2864 - acc: 0.9088 - val_loss: 0.7919 - val_acc: 0.8236
Epoch 7/10
18592/18592 [==============================] - 309s - loss: 0.2424 - acc: 0.9249 - val_loss: 0.6999 - val_acc: 0.8434
Epoch 8/10
18592/18592 [==============================] - 300s - loss: 0.2422 - acc: 0.9236 - val

In [23]:
model.save_weights(path+'models/convpool.h5')

In [24]:
model.optimizer.lr=0.00001


In [25]:

model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
18592/18592 [==============================] - 306s - loss: 0.1603 - acc: 0.9519 - val_loss: 0.7678 - val_acc: 0.8038
Epoch 2/2
18592/18592 [==============================] - 302s - loss: 0.1545 - acc: 0.9528 - val_loss: 0.8127 - val_acc: 0.8129


In [26]:
model.save_weights(path+'models/convpool2.h5')

## Imagenet Conv Features

Use pre-trained VGG weights to help

So we can pre-compute the output of the last convolutional layer (see lesson 3)  

However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.

In [7]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [6]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [9]:
conv_model = Sequential(conv_layers)

In [8]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]            
___________________________________________________________________________________________

In [10]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18975 images belonging to 10 classes.
Found 3449 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


### pre-predict the stuff from VGG Conv layers

In [11]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)

In [12]:
save_array(path+'results/conv_feat.dat', conv_feat)

In [13]:
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [14]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)

In [15]:
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [16]:
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [10]:
conv_feat = load_array(path+'results/conv_feat.dat')

In [17]:
conv_feat.shape

(18975, 512, 14, 14)

In [11]:
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [18]:
conv_val_feat.shape

(3449, 512, 14, 14)

## Batchnorm dense layers on pretrained conv layers

We've pre-computed the output of the last convolutional layer, need to create a network that takes that as input and predict our 10 classes.

We can try a simple version of VGG's dense layers


### TO DO HERE

remember to save the weights!!!!!!!!

In [26]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10,activation='softmax')
    ]

In [29]:
def get_bn2_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10,activation='softmax')
    ]

In [27]:
p=0.5

In [28]:
bn_layers = get_bn_layers(p)

In [29]:
## build a final model with VGG conv_layers and our own bn_layers
final_model = Sequential (conv_layers)
for layer in final_model.layers: layer.trainable = False
for layer in bn_layers: final_model.add(layer)

In [33]:
## this is if I don't want to train with the combined conv_layers with bn_layers
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
final_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
                                                                   lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[1][0]            
___________________________________________________________________________________________

In [30]:
final_model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 18592 images belonging to 10 classes.


In [18]:
lr=Adam(lr=10e-5)

In [31]:
final_model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
18592/18592 [==============================] - 586s - loss: 1.8956 - acc: 0.4160 - val_loss: 0.9720 - val_acc: 0.7119
Epoch 2/5
18592/18592 [==============================] - 586s - loss: 0.6649 - acc: 0.7842 - val_loss: 0.7367 - val_acc: 0.7646
Epoch 3/5
18592/18592 [==============================] - 586s - loss: 0.3507 - acc: 0.8969 - val_loss: 0.6756 - val_acc: 0.7868
Epoch 4/5
18592/18592 [==============================] - 586s - loss: 0.2228 - acc: 0.9424 - val_loss: 0.6647 - val_acc: 0.7923
Epoch 5/5
18592/18592 [==============================] - 586s - loss: 0.1604 - acc: 0.9598 - val_loss: 0.6102 - val_acc: 0.7962


In [32]:
final_model.save_weights(path+'models/vgg_conv_no_aug_data_d05.h5')

In [35]:
final_model.fit_generator(da_batches, da_batches.nb_sample, nb_epoch=1, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/1
18592/18592 [==============================] - 586s - loss: 0.2120 - acc: 0.9349 - val_loss: 0.5479 - val_acc: 0.8338


In [36]:
final_model.save_weights(path+'models/vgg_conv_with_aug_data_d05.h5')

In [14]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1,
            validation_data=(conv_val_feat, val_labels))

NameError: name 'conv_feat' is not defined

In [23]:
bn_model.optimizer.lr=0.01

In [27]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=20,
            validation_data=(conv_val_feat, val_labels))

Train on 18975 samples, validate on 3449 samples
Epoch 1/20
18975/18975 [==============================] - 8s - loss: 2.2270 - acc: 0.1814 - val_loss: 2.3189 - val_acc: 0.1247
Epoch 2/20
18975/18975 [==============================] - 8s - loss: 2.2143 - acc: 0.1822 - val_loss: 2.3393 - val_acc: 0.1206
Epoch 3/20
18975/18975 [==============================] - 8s - loss: 2.2022 - acc: 0.1921 - val_loss: 2.3391 - val_acc: 0.1212
Epoch 4/20
18975/18975 [==============================] - 8s - loss: 2.1914 - acc: 0.1998 - val_loss: 2.3516 - val_acc: 0.1131
Epoch 5/20
18975/18975 [==============================] - 8s - loss: 2.1848 - acc: 0.2083 - val_loss: 2.3511 - val_acc: 0.1038
Epoch 6/20
18975/18975 [==============================] - 8s - loss: 2.1687 - acc: 0.2133 - val_loss: 2.3516 - val_acc: 0.1081
Epoch 7/20
18975/18975 [==============================] - 8s - loss: 2.1538 - acc: 0.2220 - val_loss: 2.3730 - val_acc: 0.1055
Epoch 8/20
18975/18975 [==============================] - 8s -

In [28]:
bn_model.save_weights(path+'models/conv8.h5')

## Pre-computed data augmentation + dropout

In [19]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 22424 images belonging to 10 classes.


We use those to create a dataset of convolutional features
3x bigger than the training set.

In [13]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*3)

In [20]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample)

daconv_feat2.dat is 5x the training set.
save_array(path+'results/daconv_feat2.dat', da_conv_feat)

In [14]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [21]:
save_array(path+'results/da_conv_feat2a.dat', da_conv_feat)
## with 1 batch

In [12]:
da_conv_feat = load_array(path+'results/da_conv_feat2a.dat')
## with 1 batch

In [7]:
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

combine the real training data as well in its non-augmented form

In [13]:
da_conv_feat.shape

(22424, 512, 14, 14)

In [14]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

In [15]:
save_array(path+'results/da_conv_feat_concate2x.dat', da_conv_feat)

Since we have 6x bigger of dataset.  We need to copy the labels 6 times as well.

In [16]:
trn_labels.shape

(22424, 10)

In [17]:
da_trn_labels = np.concatenate([trn_labels]*2)

In [18]:
da_trn_labels.shape

(44848, 10)

Since we pre-computed the output of the last convolutional layers from VGG 
we need to create a network that takes that as the input ad predicts our 10 classes
let's try using a simplied version of VGG's dense layer.

In [5]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [6]:
p=0.8

In [12]:
bn_model=Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 44848 samples, validate on 4484 samples
Epoch 1/4
44848/44848 [==============================] - 21s - loss: 1.8490 - acc: 0.3342 - val_loss: 2.2465 - val_acc: 0.2417
Epoch 2/4
44848/44848 [==============================] - 20s - loss: 1.7789 - acc: 0.3589 - val_loss: 2.2749 - val_acc: 0.1262
Epoch 3/4
44848/44848 [==============================] - 20s - loss: 1.7216 - acc: 0.3782 - val_loss: 2.2834 - val_acc: 0.1287
Epoch 4/4
44848/44848 [==============================] - 21s - loss: 1.6783 - acc: 0.3954 - val_loss: 2.2947 - val_acc: 0.1162


In [28]:
bn_model.optimizer.lr=0.01

In [29]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 44848 samples, validate on 4484 samples
Epoch 1/4
44848/44848 [==============================] - 21s - loss: 1.6534 - acc: 0.4054 - val_loss: 2.2938 - val_acc: 0.1184
Epoch 2/4
44848/44848 [==============================] - 21s - loss: 1.6131 - acc: 0.4180 - val_loss: 2.2959 - val_acc: 0.1151
Epoch 3/4
44848/44848 [==============================] - 21s - loss: 1.5957 - acc: 0.4224 - val_loss: 2.2986 - val_acc: 0.1140
Epoch 4/4
44848/44848 [==============================] - 21s - loss: 1.5758 - acc: 0.4292 - val_loss: 2.2982 - val_acc: 0.1140


In [30]:
bn_model.optimizer.lr=0.0001

In [32]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=10, 
            validation_data=(conv_val_feat, val_labels))

Train on 44848 samples, validate on 4484 samples
Epoch 1/10
44848/44848 [==============================] - 21s - loss: 1.5107 - acc: 0.4486 - val_loss: 2.2990 - val_acc: 0.1135
Epoch 2/10
44848/44848 [==============================] - 21s - loss: 1.5021 - acc: 0.4540 - val_loss: 2.2980 - val_acc: 0.1062
Epoch 3/10
44848/44848 [==============================] - 21s - loss: 1.4902 - acc: 0.4565 - val_loss: 2.2989 - val_acc: 0.1028
Epoch 4/10
44848/44848 [==============================] - 21s - loss: 1.4814 - acc: 0.4577 - val_loss: 2.2981 - val_acc: 0.1137
Epoch 5/10
44848/44848 [==============================] - 20s - loss: 1.4741 - acc: 0.4598 - val_loss: 2.2985 - val_acc: 0.1137
Epoch 6/10
44848/44848 [==============================] - 20s - loss: 1.4722 - acc: 0.4603 - val_loss: 2.2993 - val_acc: 0.1135
Epoch 7/10
44848/44848 [==============================] - 20s - loss: 1.4621 - acc: 0.4631 - val_loss: 2.2998 - val_acc: 0.1019
Epoch 8/10
44848/44848 [==============================]

In [33]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

## Pseudo labeling
Try using a combination of pseudo labeling and knowledge distillation to allow us to use unlabeled data (semi-supervised learning).    For this experiment, we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set.

AT a later date we'll try using the test set.


To do this, we calculate the predictions of our model...

In [34]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

concatenate with the training models

In [35]:
comb_psuedo = np.concatenate([da_trn_labels, val_pseudo])

In [36]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

In [37]:
save_array(path+'results/com_feat.dat', comb_feat)

In [38]:
save_array(path+'results/comb_psuedo.dat', comb_psuedo)

In [13]:
comb_feat = load_array(path+'results/com_feat.dat')

fine-tune the model using this data

In [12]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [39]:
bn_model.fit(comb_feat, comb_psuedo, batch_size=batch_size, nb_epoch=1, 
            validation_data=(conv_val_feat, val_labels))

Train on 49332 samples, validate on 4484 samples
Epoch 1/1
49332/49332 [==============================] - 22s - loss: 1.5266 - acc: 0.4467 - val_loss: 2.2992 - val_acc: 0.1135


In [40]:
bn_model.fit(comb_feat, comb_psuedo, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 49332 samples, validate on 4484 samples
Epoch 1/4
49332/49332 [==============================] - 22s - loss: 1.5202 - acc: 0.4456 - val_loss: 2.2992 - val_acc: 0.1135
Epoch 2/4
49332/49332 [==============================] - 22s - loss: 1.5218 - acc: 0.4438 - val_loss: 2.2985 - val_acc: 0.1135
Epoch 3/4
49332/49332 [==============================] - 23s - loss: 1.5160 - acc: 0.4489 - val_loss: 2.2990 - val_acc: 0.0999
Epoch 4/4
49332/49332 [==============================] - 22s - loss: 1.5158 - acc: 0.4449 - val_loss: 2.2986 - val_acc: 0.1135


In [41]:
bn_model.optimizer.lr=0.00001

In [42]:
bn_model.fit(comb_feat, comb_psuedo, batch_size=batch_size, nb_epoch=4, 
            validation_data=(conv_val_feat, val_labels))

Train on 49332 samples, validate on 4484 samples
Epoch 1/4
49332/49332 [==============================] - 23s - loss: 1.5114 - acc: 0.4490 - val_loss: 2.2990 - val_acc: 0.1019
Epoch 2/4
49332/49332 [==============================] - 22s - loss: 1.5084 - acc: 0.4480 - val_loss: 2.2984 - val_acc: 0.1135
Epoch 3/4
49332/49332 [==============================] - 23s - loss: 1.5014 - acc: 0.4497 - val_loss: 2.2984 - val_acc: 0.1135
Epoch 4/4
49332/49332 [==============================] - 23s - loss: 1.5025 - acc: 0.4518 - val_loss: 2.3000 - val_acc: 0.1028


In [43]:
bn_model.save_weights(path+'models/bn_ps8.h5')

## Submit
clip the amount using the validation set, prior to submitting

In [14]:
bn_model.load_weights(path+'models/bn_ps8.h5')

In [28]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [22]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

NameError: name 'val_preds' is not defined

In [31]:
## using the 4 conv/maxpool + dropout model.
##model.load_weights(path+'models/convpool.h5')
preds_gen = model.predict_generator(test_batches, test_batches.nb_sample)

In [32]:
save_array(path+'results/preds_gen_feat.dat', preds_gen)

In [39]:
## using the vgg.
##model.load_weights(path+'models/vgg_conv_with_aug_data_d05.h5')
preds_gen = final_model.predict_generator(test_batches, test_batches.nb_sample)

In [40]:
subm = do_clip(preds_gen, 0.93)

In [17]:
conv_test_feat=load_array(path+'results/conv_test_feat.dat')

In [18]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [29]:
subm = do_clip(preds,0.93)

In [41]:
subm_name = path + 'results/subm_preds_gen_da.gz'

In [45]:
classes = sorted(batches.class_indices, key = batches.class_indices.get)

In [48]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.044872,0.044872,0.044872,0.044872,0.044872,0.044872,0.044872,0.044872,0.044872,0.596154
1,img_14887.jpg,0.201163,0.048040,0.048040,0.048040,0.048040,0.048040,0.048040,0.048040,0.050882,0.411678
2,img_62885.jpg,0.045058,0.045058,0.045058,0.045058,0.594482,0.045058,0.045058,0.045058,0.045058,0.045058
3,img_45125.jpg,0.045108,0.045108,0.045108,0.045108,0.045108,0.045108,0.083949,0.045108,0.555191,0.045108
4,img_22633.jpg,0.069827,0.093062,0.056733,0.056733,0.056733,0.070164,0.056733,0.072139,0.118522,0.349353


In [49]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [50]:
FileLink(subm_name)

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data/results/subm_preds_gen_da.gz